In [ ]:
# Pseudo code for the SVM SMO algorithm
# Algorithm 6.2 SMO with Maximum Violating Pair working set selection
# 1:  for all values {1 . . . n} αk  <-- 0 ⊲ Initial coefficients
# 2:  for all values {1 . . . n} gk  <-- 1 ⊲ Initial gradient
# 3:  loop
# 4:  i <-- argmaxi yigi subject to yiαi < Bi
# 5:  j <-- argminj yjgj subject to Aj < yjαj ⊲ Maximal violating pair
# 6:  if yigi <= yjgj stop. ⊲ Optimality criterion
# 7:  λ <- min{ Bi − yiαi, yjαj − Aj , yigi − yjgj Kii + Kjj − 2Kij } ⊲ Direction search
# 8:  for all values {1 . . . n} gk <-- gk − λyk Kik + λyk Kjk ⊲ Update gradient
# 9:  αi <-- αi + yiλ
# 10: αj <-- αj − yjλ ⊲ Update coefficients
# 11: end loop

import numpy as np
import datetime as dt
from sklearn import metrics
import cffi
from pynq import MMIO
from pynq import Overlay
from pynq import PL
import pynq.lib.dma as DMA
from time import sleep, time
from pynq import Xlnk
xlnk = Xlnk()
ffi = cffi.FFI()

In [ ]:
batch_size = 6
class_size = 10
FEAT = 784

# global arrays for support vectors, dual coefficients and estimators per class
support_vectors_temp_x = np.empty((class_size), dtype = object)
dual_coef_temp = np.empty((class_size), dtype = object)
estimators_temp = np.empty((class_size), dtype = object)

In [ ]:
# MMIO addresses for our accelerator IP core
ACCEL_CTRL = 0x43C00000
mmult_ip = MMIO(ACCEL_CTRL,0x10000)

In [ ]:
# Function to be able to switch between bitstreams of Kernel function and predict fucntion IPs
def bitstream_switcher(flag):    
    if(flag == 1):    
        global index_dma, input_dma, output_dma
        # Download the custom overlay
        ol = Overlay("accelerator.bit")
        ol.download()

        index_dma = ol.axi_dma_0
        input_dma = ol.axi_dma_1
        output_dma = ol.axi_dma_5        

    if(flag == 2):        
        global dma1, dma2, dma3, dma4
        # Download the custom overlay
        ol = Overlay("accelerator.bit")
        ol.download()

        # Initialize DMA1 (mem to FPGA)
        dma1 = ol.axi_dma_2
        dma2 = ol.axi_dma_3
        dma3 = ol.axi_dma_4
        dma4 = ol.axi_dma_6

In [ ]:
class kernels(object):
    @staticmethod
    def rbf(gamma):
        def f(x, y):
            exponent = - gamma * np.linalg.norm(x-y) ** 2
            return np.exp(exponent)
        return f

In [ ]:
# hardware implementation call of the kernel matrix function
def kernel_matrix_hw(input_array,index_array):
    global index_dma, input_dma, output_dma
    
    # determine tile_sizes
    remainder = int(input_array.shape[0] % 2500)
    iterations = int(input_array.shape[0] / 2500)
        
    # final output array after batch processing
    output_array = np.empty((input_array.shape[0]), dtype=np.float32)
    
    # after determining number of batch iterations needed, loopcounter as loop variable.
    for loopcounter in range(iterations):
        # can handle only 2500 values at a time, so 2500 * 4 + 10000
        length_x = 2500
        tile_size = int(length_x / 100)
        remaining_size = length_x % 100  
        in_buffer_index = xlnk.cma_array(shape=(FEAT), dtype=np.float32)
        in_buffer_kernel = xlnk.cma_array(shape=(length_x*FEAT), dtype=np.float32)  
        out_buffer_kernel = xlnk.cma_array(shape=(length_x), dtype=np.float32)  
        
        # instantiate pre written values to the IPs registers before accelerator call
        mmult_ip.write(0x10, 1) # function flag
        mmult_ip.write(0x18, length_x) # number of samples
        mmult_ip.write(0x20, tile_size) # tile size 
        mmult_ip.write(0x28, remaining_size) # remaining tile size

        ctrl=mmult_ip.read(0x00)&0x08 # Start the accelerator
        mmult_ip.write(0x00, (ctrl|0x81))
        
        # move values to memory buffer for DMa transfer
        ffi.memmove(in_buffer_index, ffi.cast("uint32_t *", index_array.ctypes.data), FEAT*4)
        ffi.memmove(in_buffer_kernel, ffi.cast("uint32_t *", input_array[loopcounter*2500 : ((loopcounter+1)*2500)].ctypes.data), length_x*FEAT*4)
        
        # transfer values to DMA buffer
        index_dma.sendchannel.transfer(in_buffer_index)
        input_dma.sendchannel.transfer(in_buffer_kernel)
        output_dma.recvchannel.transfer(out_buffer_kernel)
        
        # handshake signals to determine completion of DMA send and receives
        index_dma.sendchannel.wait()
        input_dma.sendchannel.wait()
        output_dma.recvchannel.wait()
        
        # clean allocated buffers and connections between DMA and kernel after transaction complete
        in_buffer_index.close()
        in_buffer_kernel.close()
        out_buffer_kernel.close()
        
        output_array[loopcounter*2500 : ((loopcounter+1)*2500)] = out_buffer_kernel;
    
    # for the remaining number of values that might not be a value of mod 2500
    if(remainder > 0):
        length_x = remainder
        tile_size = int(length_x / 100)
        remaining_size = length_x % 100  
        
        in_buffer_index = xlnk.cma_array(shape=(FEAT), dtype=np.float32)
        in_buffer_kernel = xlnk.cma_array(shape=(length_x*FEAT), dtype=np.float32)  
        out_buffer_kernel = xlnk.cma_array(shape=(length_x), dtype=np.float32)  

        mmult_ip.write(0x10, 1) # function flag
        mmult_ip.write(0x18, length_x) # number of samples
        mmult_ip.write(0x20, tile_size) # tile size 
        mmult_ip.write(0x28, remaining_size) # remaining tile size

        ctrl=mmult_ip.read(0x00)&0x08 # Start the accelerator
        mmult_ip.write(0x00, (ctrl|0x81))

        ffi.memmove(in_buffer_index, ffi.cast("uint32_t *", index_array.ctypes.data), FEAT*4)
        ffi.memmove(in_buffer_kernel, ffi.cast("uint32_t *", input_array[(input_array.shape[0] - remainder) : (input_array.shape[0])].ctypes.data), length_x*FEAT*4)

        index_dma.sendchannel.transfer(in_buffer_index)
        input_dma.sendchannel.transfer(in_buffer_kernel)
        output_dma.recvchannel.transfer(out_buffer_kernel)

        index_dma.sendchannel.wait()
        input_dma.sendchannel.wait()
        output_dma.recvchannel.wait()

        in_buffer_index.close()
        in_buffer_kernel.close()
        out_buffer_kernel.close()
        
        output_array[(input_array.shape[0] - remainder) : (input_array.shape[0])] = out_buffer_kernel; 
        
    return output_array

In [ ]:
def predict_function_hw(input_array1, input_array2, input_array3):
    global dma1, dma2, dma3, dma4
    
    iterations_predict = int(input_array1.shape[0] / 100)
    remainder_predict = int(input_array1.shape[0] % 100)
    
    output_array_predict = np.empty((input_array1.shape[0]), dtype=np.float32)
    
    for loopcounter_predict in range(iterations_predict):
        length_x = 100
        length_dc = input_array3.shape[0]
        
        in_buffer1 = xlnk.cma_array(shape=(length_x*FEAT), dtype=np.float32)
        in_buffer2 = xlnk.cma_array(shape=(length_dc*FEAT), dtype=np.float32)
        in_buffer3 = xlnk.cma_array(shape=(length_dc), dtype=np.float32)
        out_buffer = xlnk.cma_array(shape=(length_x), dtype=np.float32)
        
        tile_count = int(input_array2.shape[0] / 20);
        remaining = input_array2.shape[0] % 20;
        
        mmult_ip.write(0x10, 2) # function flag
        mmult_ip.write(0x30, tile_count)
        mmult_ip.write(0x38, remaining)
        mmult_ip.write(0x40, length_dc)
        
         # Start the accelerator
        ctrl=mmult_ip.read(0x00)&0x08
        mmult_ip.write(0x00, (ctrl|0x81))
        
        ffi.memmove(in_buffer1, ffi.cast("uint32_t *", input_array1[(loopcounter_predict * 100) : ((loopcounter_predict + 1)*100)-1].ctypes.data), length_x*FEAT*4)
        ffi.memmove(in_buffer2, ffi.cast("uint32_t *", input_array2.ctypes.data), length_dc*FEAT*4)
        ffi.memmove(in_buffer3, ffi.cast("uint32_t *", input_array3.ctypes.data), length_dc*4)
        
        dma1.sendchannel.transfer(in_buffer1)
        dma2.sendchannel.transfer(in_buffer2)
        dma3.sendchannel.transfer(in_buffer3)

        dma4.recvchannel.transfer(out_buffer)
        dma1.sendchannel.wait()
        dma2.sendchannel.wait()
        dma3.sendchannel.wait()
        dma4.recvchannel.wait()

        in_buffer1.close()
        in_buffer2.close()
        in_buffer3.close()
        out_buffer.close()
        
        output_array_predict[loopcounter_predict*100 : ((loopcounter_predict+1)*100)] = out_buffer;
        
    if(remainder_predict > 0):
        length_x = remainder_predict
        length_dc = input_array3.shape[0]
        
        in_buffer1 = xlnk.cma_array(shape=(length_x*FEAT), dtype=np.float32)
        in_buffer2 = xlnk.cma_array(shape=(length_dc*FEAT), dtype=np.float32)
        in_buffer3 = xlnk.cma_array(shape=(length_dc), dtype=np.float32)
        out_buffer = xlnk.cma_array(shape=(length_x), dtype=np.float32)
        
        mmult_ip.write(0x10, 2) # function flag
        mmult_ip.write(0x30, tile_count)
        mmult_ip.write(0x38, remaining)
        mmult_ip.write(0x40, length_dc)
        
         # Start the accelerator
        ctrl=mmult_ip.read(0x00)&0x08
        mmult_ip.write(0x00, (ctrl|0x81))
        
        ffi.memmove(in_buffer1, ffi.cast("uint32_t *", input_array1[(input_array1.shape[0] - remainder_predict) : input_array1.shape[0]].ctypes.data), length_x*FEAT*4)
        ffi.memmove(in_buffer2, ffi.cast("uint32_t *", input_array2.ctypes.data), length_dc*FEAT*4)
        ffi.memmove(in_buffer3, ffi.cast("uint32_t *", input_array3.ctypes.data), length_dc*4)
        
        dma1.sendchannel.transfer(in_buffer1)
        dma2.sendchannel.transfer(in_buffer2)
        dma3.sendchannel.transfer(in_buffer3)

        dma4.recvchannel.transfer(out_buffer)
        dma1.sendchannel.wait()
        dma2.sendchannel.wait()
        dma3.sendchannel.wait()
        dma4.recvchannel.wait()

        in_buffer1.close()
        in_buffer2.close()
        in_buffer3.close()
        out_buffer.close()
        
        output_array_predict[(output_array_predict.shape[0] - remainder_predict) : output_array_predict.shape[0]] = out_buffer;

    return output_array_predict

In [ ]:
class binary_svm(object):
    def __init__(self, kernel, tol, C, support_vector_tol = 0.0, max_iter = 1000):
        self.max_iter = max_iter
        self.tol = tol
        self.kernel = kernel
        self.C = C
        self.support_vector_indices = 0.0
        self.support_vectors = []
        self.dual_coef = []
        self.support_vector_tol = support_vector_tol

    def kernel_matrix(self, X, index):
        result = np.zeros(X.shape[0])
        x_i = X[index, :]
        for j, x_j in enumerate(X):
            result[j] = self.kernel(x_i, x_j)
        return result

    def smo(self, X, y):
        iterations = 0
        n_samples = X.shape[0]
        # Initial coefficients
        alpha = np.zeros(n_samples)
        # Initial gradient
        g = np.ones(n_samples)
        while True:
            yg = g * y
            # KKT Conditions
            y_pos_ind = (y == 1)
            y_neg_ind = (np.ones(n_samples) - y_pos_ind).astype(bool)
            alpha_pos_ind = (alpha >= self.C)
            alpha_neg_ind = (alpha <= 0)

            indices_violating_Bi_1 = y_pos_ind * alpha_pos_ind
            indices_violating_Bi_2 = y_neg_ind * alpha_neg_ind
            indices_violating_Bi = indices_violating_Bi_1 + indices_violating_Bi_2
            yg_i = yg.copy()
            yg_i[indices_violating_Bi] = float('-inf')
            # First of the maximum violating pair
            i = np.argmax(yg_i)
#             Kik = self.kernel_matrix(X, i)
            Kik = kernel_matrix_hw(X.astype(np.float32), X[i].astype(np.float32))

            indices_violating_Ai_1 = y_pos_ind * alpha_neg_ind
            indices_violating_Ai_2 = y_neg_ind * alpha_pos_ind
            indices_violating_Ai = indices_violating_Ai_1 + indices_violating_Ai_2
            yg_j = yg.copy()
            yg_j[indices_violating_Ai] = float('+inf')
            # Second of the maximum violating pair
            j = np.argmin(yg_j)
#             Kjk = self.kernel_matrix(X, j)
            Kjk = kernel_matrix_hw(X.astype(np.float32), X[j].astype(np.float32))

            # Optimality criterion
            if(yg_i[i] - yg_j[j]) < self.tol or (iterations >= self.max_iter):
                break

            min_term_1 = (y[i] == 1) * self.C - y[i] * alpha[i]
            min_term_2 = y[j] * alpha[j] + (y[j] == -1) * self.C
            min_term_3 = (yg_i[i] - yg_j[j]) / (Kik[i] + Kjk[j] - 2 * Kik[j])

            # Direction search
            lamda = np.min([min_term_1, min_term_2, min_term_3])
            # Gradient update
            g += lamda * y * (Kjk - Kik)
            # Update coefficients
            alpha[i] = alpha[i] + y[i] * lamda
            alpha[j] = alpha[j] - y[j] * lamda

            iterations += 1

        print('{} iterations to arrive at the minimum'.format(iterations))
        return alpha

    def train(self, X, y, label, batch_number):
        global support_vectors_temp_x, dual_coef_temp
        lagrange_multipliers = self.smo(X, y)
        self.support_vector_indices = lagrange_multipliers > self.support_vector_tol
        self.dual_coef = lagrange_multipliers[self.support_vector_indices] * y[self.support_vector_indices]
        self.support_vectors = X[self.support_vector_indices]

        if(batch_number == 0):
            support_vectors_temp_x[label] = self.support_vectors
            dual_coef_temp[label] = self.dual_coef
            print('{} Support Vectors'.format(self.support_vectors.shape[0]))

        if(batch_number != 0):
            print('{} Support Vectors'.format(self.support_vectors.shape[0]))
            self.support_vectors = np.row_stack((self.support_vectors, support_vectors_temp_x[label]))
            self.dual_coef = np.concatenate((self.dual_coef, dual_coef_temp[label]))
            support_vectors_temp_x[label] = self.support_vectors
            dual_coef_temp[label] = self.dual_coef
            print('{} Updated Support Vectors'.format(self.support_vectors.shape[0]))

    def predict(self, X):
        n_samples = X.shape[0]
        prediction = np.zeros(n_samples)
        for i, x in enumerate(X):
            result = 0.0
            for z_i, x_i in zip(self.dual_coef, self.support_vectors):
                result += z_i * self.kernel(x_i, x)
            prediction[i] = result
        return prediction
    
    def predict_hw(self, X):
        prediction = predict_function_hw(X.astype(np.float32), self.support_vectors.astype(np.float32), self.dual_coef.astype(np.float32))
        return prediction

In [ ]:
class multiclass(object):
    def __init__(self, kernel, tol, C, max_iter = 1000):
        self.max_iter = max_iter
        self.tol = tol
        self.C = C
        self.kernel = kernel
        self.indices = []
    
    # for use in the binarz SVM function
    def binarize_labels(self, X, y, label):
        pos_class_indices = (y == label)
        neg_class_indices = (y != label)
        X_binarized = X.copy()
        y_binarized = y.copy()
        y_binarized[pos_class_indices] = +1
        y_binarized[neg_class_indices] = -1
        return X_binarized, y_binarized

    def train(self, X, y, batch_number):
        global estimators_temp
        self.classes = set(y)
        classifiers = np.empty(len(self.classes), dtype = object)

        for i, label in enumerate(self.classes):
            print("Label: {}".format(label))
            svm_binary = binary_svm(kernel = self.kernel, C = self.C, max_iter = self.max_iter, tol = self.tol)
            X_binarized, y_binarized = self.binarize_labels(X, y, i)
            svm_binary.train(X_binarized, y_binarized, int(label), batch_number)
            classifiers[i] = svm_binary
            estimators_temp[int(label)] = classifiers[i]
            classifiers[i] = estimators_temp[int(label)]
        self.estimators = estimators_temp

    def predict(self, X, input):
        classes = set(input)
        n_samples = X.shape[0]
        predicted_scores = np.zeros((n_samples, len(classes)))
        for i, label in enumerate(classes):
            print("Predicting label: {}".format(i))
#             predicted_scores[:,i] = self.estimators[i].predict(X)
            predicted_scores[:,i] = self.estimators[i].predict_hw(X)

        return np.argmax(predicted_scores, axis = 1)

In [ ]:
if __name__ == "__main__":   
    
    classifier = multiclass(C = 10.0, kernel=kernels.rbf(0.05), tol = 2.0)
    start_time = dt.datetime.now()
    print('Start learning at: {}'.format(str(start_time)))

    for batch_number in range(batch_size):
        print('\nBatch number: {}'.format(batch_number))
        if(batch_number == 0):
            bitstream_switcher(1)
            X_train_batch = np.load('data/X_train_batch_{}.npy'.format(batch_number))
            y_train_batch = np.load('data/y_train_batch_{}.npy'.format(batch_number))
            classifier.train(X_train_batch, y_train_batch, batch_number)
            print(X_train_batch.shape, y_train_batch.shape)
        if(batch_number != 0):
            bitstream_switcher(2)            
            X_train_batch = np.load('data/X_train_batch_{}.npy'.format(batch_number))
            y_train_batch = np.load('data/y_train_batch_{}.npy'.format(batch_number))
            predicted = classifier.predict(X_train_batch, y_train_batch)
            bitstream_switcher(1) 
            expected = y_train_batch
            m = expected != predicted
            print('Misclassified samples: {}'.format(X_train_batch[m].shape[0]))
            classifier.train(X_train_batch[m], y_train_batch[m], batch_number)
            print(X_train_batch.shape, y_train_batch.shape)
    end_time = dt.datetime.now()
    print('Stop learning {}'.format(str(end_time)))
    elapsed_time = end_time - start_time
    print('Elapsed learning time: {}'.format(str(elapsed_time)))

    X_test = np.load('data/X_test.npy')
    y_test = np.load('data/y_test.npy')
    expected = y_test
    start_time = dt.datetime.now()
    print('Start predicting at: {}'.format(str(start_time)))
    predicted = classifier.predict(X_test, np.arange(estimators_temp.shape[0]))
    end_time = dt.datetime.now()
    print('Stop predicting: {}'.format(str(end_time)))
    elapsed_time = end_time - start_time
    print('Elapsed predicting time: {}'.format(str(elapsed_time)))

    print("Classification report:\n%s\n" % (metrics.classification_report(expected, predicted)))
    cm = metrics.confusion_matrix(expected, predicted)
    print("Confusion matrix:\n%s" % cm)
    print("Accuracy: {}".format(metrics.accuracy_score(expected, predicted)*100))